In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from signal_dataset import SignalDataset
from sklearn.model_selection import train_test_split
import re
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

In [20]:
sr = 1562500
signal_data_dir = "/home/petr/Documents/Motor_project/AE_PETR_motor/"
bin_setup = [{"label": i.stem, "channels": len(list(i.glob('*.bin'))), "interval": [0, 15 * sr], "bin_path": list(i.glob('*.bin'))[0]} for i in
             Path(signal_data_dir).glob('WUP*') if re.search(r'\d$', i.stem)]

sd = SignalDataset(step=1000, window_size=1000, bin_setup=bin_setup, device="cpu", source_dtype="float32", sklearn=True)

In [21]:
bin_setup

[{'label': 'WUP_20',
  'channels': 1,
  'interval': [0, 23437500],
  'bin_path': PosixPath('/home/petr/Documents/Motor_project/AE_PETR_motor/WUP_20/AE-DATA-conti-7392894521898843-ch1.bin')},
 {'label': 'WUP_50',
  'channels': 1,
  'interval': [0, 23437500],
  'bin_path': PosixPath('/home/petr/Documents/Motor_project/AE_PETR_motor/WUP_50/AE-DATA-conti-7392894547392940-ch1.bin')},
 {'label': 'WUP_10',
  'channels': 1,
  'interval': [0, 23437500],
  'bin_path': PosixPath('/home/petr/Documents/Motor_project/AE_PETR_motor/WUP_10/AE-DATA-conti-7392894509828124-ch1.bin')},
 {'label': 'WUP_90',
  'channels': 1,
  'interval': [0, 23437500],
  'bin_path': PosixPath('/home/petr/Documents/Motor_project/AE_PETR_motor/WUP_90/AE-DATA-conti-7392894585243287-ch1.bin')},
 {'label': 'WUP_60',
  'channels': 1,
  'interval': [0, 23437500],
  'bin_path': PosixPath('/home/petr/Documents/Motor_project/AE_PETR_motor/WUP_60/AE-DATA-conti-7392894554399074-ch1.bin')},
 {'label': 'WUP_30',
  'channels': 1,
  'inte

In [4]:
from torch.utils.data import random_split
train_data, test_data = random_split(sd, [0.8, 0.2])

In [5]:
X_train = np.asarray([data[0] for data in train_data]) 
y_train = np.asarray([data[1] for data in train_data]) 
X_test = np.asarray([data[0] for data in test_data]) 
y_test = np.asarray([data[1] for data in test_data]) 

In [6]:
n_components = 100  # Adjust this as needed
pca = PCA(n_components=n_components)

# Fit PCA to the training data
pca.fit(X_train)

PCA(n_components=100)

In [7]:
X_train.shape

(168747, 1000)

In [ ]:
model = SVC(kernel="linear")
model.fit(X_train, y_train) 

In [ ]:
y_pred = model.predict(X_train)

In [ ]:


# Evaluate accuracy
creport = classification_report(y_train, y_pred)
print(creport)